In [203]:
import numpy as np
import math
import pandas as pd
import seaborn as sns
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as seabornInstance 

import statsmodels.api as sm
from statsmodels.stats import diagnostic as diag
from statsmodels.stats.outliers_influence import variance_inflation_factor

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics

%matplotlib inline

In [204]:
# load csv data into a dataframe (df) and select the top 5000 rows
df = pd.read_csv('train.csv')
validation_df = pd.read_csv('validation.csv')
df = df[:5000]
validation_df = validation_df[:1000]

#remove column white space
df.columns = df.columns.str.replace(' ', '')
validation_df.columns = validation_df.columns.str.replace(' ', '')

#drop url column
df.drop('url', axis=1, inplace=True)
validation_df.drop('url', axis=1, inplace=True)


# set the orignal dataframe index to the id column
df = df.set_index('id')
validation_df = validation_df.set_index('id')


# Make sure all data is float
df = df.astype(float)
validation_df = validation_df.astype(float)
        
predictions_df = validation_df.copy()

validation_df = validation_df.drop('shares', axis = 1)
df

/Users/diogovalentepcs/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,timedelta,n_tokens_title,n_tokens_content,n_unique_tokens,n_non_stop_words,n_non_stop_unique_tokens,num_hrefs,num_self_hrefs,num_imgs,num_videos,...,min_positive_polarity,max_positive_polarity,avg_negative_polarity,min_negative_polarity,max_negative_polarity,title_subjectivity,title_sentiment_polarity,abs_title_subjectivity,abs_title_sentiment_polarity,shares
id,,,,,,,,,,,,,,,,,,,,,
1.0,198.0,6.0,47.0,0.914894,1.0,0.964286,1.0,1.0,0.0,2.0,...,0.350000,0.350,-0.200000,-0.200000,-0.200000,0.443939,-0.015152,0.056061,0.015152,1170.0
2.0,660.0,7.0,181.0,0.519337,1.0,0.644231,5.0,2.0,1.0,0.0,...,0.136364,1.000,-0.400000,-0.400000,-0.400000,0.000000,0.000000,0.500000,0.000000,6265.0
3.0,552.0,9.0,862.0,0.465089,1.0,0.635478,16.0,0.0,1.0,0.0,...,0.050000,1.000,-0.264444,-0.750000,-0.100000,0.000000,0.000000,0.500000,0.000000,121.0
4.0,559.0,10.0,1015.0,0.447503,1.0,0.636986,12.0,2.0,1.0,0.0,...,0.100000,1.000,-0.277083,-0.800000,-0.008333,0.000000,0.000000,0.500000,0.000000,841.0
5.0,573.0,8.0,129.0,0.666667,1.0,0.790123,3.0,0.0,1.0,0.0,...,0.100000,0.375,-0.393333,-0.700000,-0.166667,0.000000,0.000000,0.500000,0.000000,376.0
6.0,149.0,9.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,2.0,...,0.000000,0.000,0.000000,0.000000,0.000000,1.000000,0.500000,0.500000,0.500000,16034.0
7.0,702.0,8.0,234.0,0.595652,1.0,0.746377,12.0,4.0,1.0,0.0,...,0.500000,0.500,-0.135714,-0.200000,-0.071429,0.300000,0.250000,0.200000,0.250000,2116.0
8.0,111.0,10.0,437.0,0.527907,1.0,0.708502,5.0,2.0,2.0,0.0,...,0.100000,0.600,-0.314286,-1.000000,-0.050000,0.687500,-0.062500,0.187500,0.062500,1577.0
9.0,709.0,10.0,289.0,0.614035,1.0,0.769231,6.0,1.0,1.0,0.0,...,0.136364,0.600,-0.312500,-0.500000,-0.125000,0.727273,0.068182,0.227273,0.068182,961.0


In [205]:
# Defining Dependent Variable (Y) and Inependent Variables (x)
X = df.drop('shares', axis = 1)
Y = df[['shares']]

# Create models' dataframe with same variables as originals but empty
models = X.iloc[0:0]

# Add column for the constant (on first column)
models.insert(0, 'const', [], True)
models

,const,timedelta,n_tokens_title,n_tokens_content,n_unique_tokens,n_non_stop_words,n_non_stop_unique_tokens,num_hrefs,num_self_hrefs,num_imgs,...,avg_positive_polarity,min_positive_polarity,max_positive_polarity,avg_negative_polarity,min_negative_polarity,max_negative_polarity,title_subjectivity,title_sentiment_polarity,abs_title_subjectivity,abs_title_sentiment_polarity


In [206]:
# Costumize independet variables
def useVar(variables_to_use):
    for column in X.columns:
        if column not in variables_to_use:
            X.drop(column, axis=1, inplace=True)
useVar(['kw_avg_avg', 'LDA_03','weekday_is_saturday', 'data_channel_is_world', 'num_imgs', 'n_tokens_title', 'n_unique_tokens'])
X

,n_tokens_title,n_unique_tokens,num_imgs,data_channel_is_world,kw_avg_avg,weekday_is_saturday,LDA_03
id,,,,,,,
1.0,6.0,0.914894,0.0,0.0,3714.512814,0.0,0.386811
2.0,7.0,0.519337,1.0,0.0,2498.783924,0.0,0.066743
3.0,9.0,0.465089,1.0,1.0,2543.161561,0.0,0.170934
4.0,10.0,0.447503,1.0,0.0,2572.800545,0.0,0.020318
5.0,8.0,0.666667,1.0,0.0,2029.340699,0.0,0.030580
6.0,9.0,0.000000,0.0,0.0,5069.770031,0.0,0.370077
7.0,8.0,0.595652,1.0,0.0,4744.522336,0.0,0.463240
8.0,10.0,0.527907,2.0,1.0,3159.607252,0.0,0.050343
9.0,10.0,0.614035,1.0,0.0,1440.709455,0.0,0.040001


In [207]:
# Transform variables
def logVar(var):
    if var != 0:
        var = np.log(var)
        
        

X.apply(lambda x: logVar(x['num_imgs']),axis=1)
X.apply(lambda x: logVar(x['kw_avg_avg']),axis=1)
X

,n_tokens_title,n_unique_tokens,num_imgs,data_channel_is_world,kw_avg_avg,weekday_is_saturday,LDA_03
id,,,,,,,
1.0,6.0,0.914894,0.0,0.0,3714.512814,0.0,0.386811
2.0,7.0,0.519337,1.0,0.0,2498.783924,0.0,0.066743
3.0,9.0,0.465089,1.0,1.0,2543.161561,0.0,0.170934
4.0,10.0,0.447503,1.0,0.0,2572.800545,0.0,0.020318
5.0,8.0,0.666667,1.0,0.0,2029.340699,0.0,0.030580
6.0,9.0,0.000000,0.0,0.0,5069.770031,0.0,0.370077
7.0,8.0,0.595652,1.0,0.0,4744.522336,0.0,0.463240
8.0,10.0,0.527907,2.0,1.0,3159.607252,0.0,0.050343
9.0,10.0,0.614035,1.0,0.0,1440.709455,0.0,0.040001


In [208]:
# Iteration for regression creation
for i in range(0,1000):
    X = sm.add_constant(X) # necessary for code to work    

    # Split X and Y into train and test parts (uso 20% dos dados para calcular/ treinar a regressão e testo em 80% resto, a seleção é aleatória)
    x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.80)


    #Fit a linear model using Ordinary Least Squares
    regression_model = sm.OLS(y_train, x_train)

    #fit the data
    est = regression_model.fit()
    
    # predict values
    y_pred = est.predict(x_test)
    
    # Retrieve coef and stats
    mae = metrics.mean_absolute_error(y_test, y_pred)
    mse = metrics.mean_squared_error(y_test, y_pred)
    rmse = math.sqrt(metrics.mean_squared_error(y_test, y_pred))
    r2 = est.rsquared
    adj_r2 = est.rsquared_adj
    f_pvalue = est.f_pvalue
    
    #creating coeff dictionary
    coeff = {}
    for key in est.params.keys(): #loops through all variables
        coeff[key] = est.params[key]

    # Add the metrics to the dictionary (new_model beacuse it is a row to add to models)
    new_model = coeff
    new_model['r2'] = r2
    new_model['adj_r2'] = adj_r2
    new_model['f_pvalue'] = f_pvalue
    new_model['mae'] = mae
    new_model['mse'] = mse
    new_model['rmse'] = rmse
    # Add new_model to models
    models = models.append(new_model, ignore_index=True)
models

,const,timedelta,n_tokens_title,n_tokens_content,n_unique_tokens,n_non_stop_words,n_non_stop_unique_tokens,num_hrefs,num_self_hrefs,num_imgs,...,title_subjectivity,title_sentiment_polarity,abs_title_subjectivity,abs_title_sentiment_polarity,adj_r2,f_pvalue,mae,mse,r2,rmse
0,-3171.525156,NaN,85.837391,NaN,4327.516849,NaN,NaN,NaN,NaN,53.932071,...,NaN,NaN,NaN,NaN,0.043997,6.572785e-09,3944.760502,2.362305e+09,0.050696,48603.553228
1,-5060.206474,NaN,157.707352,NaN,-314.228030,NaN,NaN,NaN,NaN,31.241939,...,NaN,NaN,NaN,NaN,0.079853,1.389243e-16,3443.182310,7.416275e+07,0.086300,8611.779767
2,-2719.002967,NaN,18.929194,NaN,2900.667424,NaN,NaN,NaN,NaN,33.971529,...,NaN,NaN,NaN,NaN,0.040718,3.063779e-08,3748.890492,1.091809e+09,0.047439,33042.531795
3,988.651203,NaN,-132.207199,NaN,825.866523,NaN,NaN,NaN,NaN,66.191825,...,NaN,NaN,NaN,NaN,0.026068,2.367163e-05,3176.452449,1.467969e+08,0.032892,12115.977810
4,-530.349818,NaN,29.529525,NaN,-0.232334,NaN,NaN,NaN,NaN,77.298534,...,NaN,NaN,NaN,NaN,0.050178,3.465840e-10,2999.815564,6.834874e+07,0.056833,8267.329871
5,-1966.831879,NaN,-29.969542,NaN,3502.893504,NaN,NaN,NaN,NaN,90.675352,...,NaN,NaN,NaN,NaN,0.051181,2.139696e-10,3694.226783,1.574773e+09,0.057830,39683.405298
6,-1136.690066,NaN,-6.581806,NaN,1674.612197,NaN,NaN,NaN,NaN,28.681885,...,NaN,NaN,NaN,NaN,0.040938,2.764325e-08,3341.530114,4.104337e+08,0.047658,20259.162125
7,-3482.931842,NaN,193.224920,NaN,2839.751263,NaN,NaN,NaN,NaN,30.146337,...,NaN,NaN,NaN,NaN,0.034500,5.415592e-07,3541.720270,1.053354e+09,0.041265,32455.410860
8,-1570.748721,NaN,256.956210,NaN,3.473871,NaN,NaN,NaN,NaN,25.653943,...,NaN,NaN,NaN,NaN,0.027741,1.132677e-05,3120.086948,6.162064e+07,0.034553,7849.881645
9,-611.944293,NaN,21.886455,NaN,2.399341,NaN,NaN,NaN,NaN,31.389340,...,NaN,NaN,NaN,NaN,0.023611,6.895629e-05,3033.023545,6.589895e+07,0.030453,8117.816879


In [223]:
final_model_dict = {}
for column in models.columns:
    final_model_dict[column] = models[column].mean()
    # change to 0 null coefficients
    if math.isnan(final_model_dict[column]):
        final_model_dict[column] = 0
final_model_dict['usedVariables'] = 'kw_avg_avg, LDA_03, weekday_is_saturday, data_channel_is_world, num_imgs, n_tokens_title, n_unique_tokens'
final_model_dict['trasnformedVariables'] = 'kw_avg_avg = ln(kw_avg_avg); num_imgs = ln(num_imgs)'
final_model_dict['uesdComponents'] = ''
final_model_dict['otherDescription'] = ''

final_model_dict

{'const': -1012.188225182101,
 'timedelta': 0,
 'n_tokens_title': 46.68366095106704,
 'n_tokens_content': 0,
 'n_unique_tokens': 1009.8429208609443,
 'n_non_stop_words': 0,
 'n_non_stop_unique_tokens': 0,
 'num_hrefs': 0,
 'num_self_hrefs': 0,
 'num_imgs': 33.65587573346167,
 'num_videos': 0,
 'average_token_length': 0,
 'num_keywords': 0,
 'data_channel_is_lifestyle': 0,
 'data_channel_is_entertainment': 0,
 'data_channel_is_bus': 0,
 'data_channel_is_socmed': 0,
 'data_channel_is_tech': 0,
 'data_channel_is_world': -291.6847534048261,
 'kw_min_min': 0,
 'kw_max_min': 0,
 'kw_avg_min': 0,
 'kw_min_max': 0,
 'kw_max_max': 0,
 'kw_avg_max': 0,
 'kw_min_avg': 0,
 'kw_max_avg': 0,
 'kw_avg_avg': 0.9533802582451686,
 'self_reference_min_shares': 0,
 'self_reference_max_shares': 0,
 'self_reference_avg_sharess': 0,
 'weekday_is_monday': 0,
 'weekday_is_tuesday': 0,
 'weekday_is_wednesday': 0,
 'weekday_is_thursday': 0,
 'weekday_is_friday': 0,
 'weekday_is_saturday': 919.9381105998849,
 'we

In [224]:
regressions = []
regressions.append(final_model_dict)

regressions_df = models.iloc[0:0]
regressions_df = regressions_df.append(final_model_dict, ignore_index=True)

In [232]:
#regressions_df.to_csv('regressions_df.csv', index=False)
#regressions_df

import json
with open('regressions.json', 'w') as outfile:
    json.dump(regressions, outfile)
    
regressions = []
with open('regressions.json') as json_file:
    regressions = json.load(json_file)
regressions

[{'const': -1012.188225182101,
  'timedelta': 0,
  'n_tokens_title': 46.68366095106704,
  'n_tokens_content': 0,
  'n_unique_tokens': 1009.8429208609443,
  'n_non_stop_words': 0,
  'n_non_stop_unique_tokens': 0,
  'num_hrefs': 0,
  'num_self_hrefs': 0,
  'num_imgs': 33.65587573346167,
  'num_videos': 0,
  'average_token_length': 0,
  'num_keywords': 0,
  'data_channel_is_lifestyle': 0,
  'data_channel_is_entertainment': 0,
  'data_channel_is_bus': 0,
  'data_channel_is_socmed': 0,
  'data_channel_is_tech': 0,
  'data_channel_is_world': -291.6847534048261,
  'kw_min_min': 0,
  'kw_max_min': 0,
  'kw_avg_min': 0,
  'kw_min_max': 0,
  'kw_max_max': 0,
  'kw_avg_max': 0,
  'kw_min_avg': 0,
  'kw_max_avg': 0,
  'kw_avg_avg': 0.9533802582451686,
  'self_reference_min_shares': 0,
  'self_reference_max_shares': 0,
  'self_reference_avg_sharess': 0,
  'weekday_is_monday': 0,
  'weekday_is_tuesday': 0,
  'weekday_is_wednesday': 0,
  'weekday_is_thursday': 0,
  'weekday_is_friday': 0,
  'weekday_

In [226]:
def modelPredict(X, model_dict):
    y_predict = []
    for row in X.index:
        y = model_dict['const']
        for column in X.columns:
            y += X[column][row]*model_dict[column]
        y_predict.append(y)
    return y_predict


final_pred = modelPredict(validation_df, regressions[0])
final_pred

[1677.773769308947,
 3252.9211733440684,
 1836.5245272797101,
 3826.708398977745,
 2914.191017129602,
 2311.801890095994,
 3383.0432723507715,
 3583.709252779096,
 2920.8176815467673,
 2537.676491130352,
 2763.928955782825,
 6358.713794571631,
 2607.948147350319,
 2739.7836184137345,
 5190.410873714209,
 4161.738078962189,
 1921.0341832063802,
 4580.449832851543,
 2164.2391179101346,
 2886.9981761723207,
 1883.2759272148069,
 5791.306256604556,
 4678.213260948743,
 2996.4910915266196,
 2234.6194369362556,
 1996.4357071641077,
 2675.5318786448797,
 2471.9340103760615,
 2939.4411888312884,
 3714.8345087096122,
 2961.919414789652,
 5403.595293176273,
 3749.7483348803435,
 3240.330768443768,
 3563.0654067935266,
 3571.339767259984,
 3533.0763388020664,
 4667.586976034207,
 5607.233923152806,
 3905.401066867379,
 4618.308278463378,
 2497.5543559493713,
 2328.2567537701734,
 2656.9006781349376,
 5028.937836867589,
 2694.095003796486,
 2916.603894722748,
 5118.308612784856,
 2042.364499020721

In [228]:
#dropping all columns
for column in predictions_df.columns:
    if column != 'shares':
        predictions_df = predictions_df.drop(column, axis = 1)
predictions_df['model 1'] = final_pred
predictions_df

,model 1
id,
5001,1677.773769
5002,3252.921173
5003,1836.524527
5004,3826.708399
5005,2914.191017
5006,2311.801890
5007,3383.043272
5008,3583.709253
5009,2920.817682


In [ ]:
#write prediction
predictions_df.to_csv('predictions.csv')